In [ ]:
!pip install datasets
!pip install evaluate
!install transformers[torch]

In [ ]:
#import library
import cv2
import numpy as np
import os
import json
import datasets
import transformers
import torch
import evaluate

In [ ]:
import cv2
import numpy as np
import os
import random
# def apply_perspective_transform(img, tilt_range=(-15, 15)):
#     # Convert tilt range from degrees to radians
#     tilt_range = [np.deg2rad(tilt) for tilt in tilt_range]

#     # Choose random tilt angles for x and y within the specified range
#     tilt_x = random.uniform(*tilt_range)
#     tilt_y = random.uniform(*tilt_range)

#     # Compute the perspective transform matrix for the tilt
#     rows, cols = img.shape[:2]
#     src_points = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])
#     print('src_point',src_points)

#     # # Calculate offset caused by tilting
#     # offset_x = cols * np.tan(tilt_y)
#     # offset_y = rows * np.tan(tilt_x)

#     # Calculate the destination points for the perspective transformation
#     dst_points = np.float32([
#         [0 + random.uniform(-0.15,0.15)*cols, 0 + random.uniform(-0.15,0.15)*rows], 
#         [cols + random.uniform(-0.15,0.15)*cols, 0 + random.uniform(-0.15,0.15)*rows], 
#         [0 + random.uniform(-0.15,0.15)*cols, rows + random.uniform(-0.15,0.15)*rows], 
#         [cols + random.uniform(-0.15,0.15)*cols, rows + random.uniform(-0.15,0.15)*rows]
#     ])
#     print('dst_point',dst_points)

#     M = cv2.getPerspectiveTransform(src_points, dst_points)

#     # Apply the perspective transform (tilt)
#     img = cv2.warpPerspective(img, M, (cols, rows))

#     return img
def rotate_and_shear_image(im, bk_height, bk_width):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    change from https://stackoverflow.com/a/51109152
    """
    # Randomize the angle, x_shear, y_shear
    size = random.uniform(0.3, 0.5)  
    angle = random.randint(0, 359)
    x_shear = random.uniform(-0.15, 0.15)
    y_shear = random.uniform(-0.15, 0.15)

    #print(im.shape)
    height, width = im.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_im = cv2.getRotationMatrix2D(image_center, angle, 1.)

    rotation_im[0,1] += x_shear
    rotation_im[1,0] += y_shear

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos_x = abs(rotation_im[0,0]) 
    abs_sin_x = abs(rotation_im[0,1])
    abs_cos_y = abs(rotation_im[1,0]) 
    abs_sin_y = abs(rotation_im[1,1])

    # find the new width and height bounds
    bound_w = int(width * abs_cos_x +  height *  abs_sin_x )
    bound_h = int(height * abs_sin_y  + width * abs_cos_y )

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_im[0, 2] += bound_w/2 - image_center[0] - image_center[1] * x_shear
    rotation_im[1, 2] += bound_h/2 - image_center[1] - image_center[0] * y_shear

    board_mode = cv2.BORDER_CONSTANT
    color = (random.randint(230,250), random.randint(230,250), random.randint(230,250))
    
    # rotate image with the new bounds and translated rotation imrix
    im = cv2.warpAffine(im, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=board_mode, borderValue=color)
    
    # Calculate the ratio size of the image, assume background width > height
    # Set document image size to 0.3 - 0.5 of the background image size
    new_height, new_width = im.shape[:2]
    ratio = size * bk_height / new_height
    final_height = int(new_height * ratio)
    final_width = int(new_width * ratio)

    # Resize the image
    im = cv2.resize(im, (final_width, final_height))
    # im = cv2.resize(im, (0.3, 0.5))

    #print(rotated_im.shape)
    return im


def apply_perspective_transform(img, tilt_range=(-0.15, 0.15)):
    # Convert tilt range from degrees to radians
    tilt_range = [np.deg2rad(tilt) for tilt in tilt_range]

    # Compute the perspective transform matrix for the tilt
    rows, cols = img.shape[:2]
    src_points = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])

    # Calculate the destination points for the perspective transformation
    dst_points = np.float32([
        [0  + cols * np.tan(random.uniform(-0.15, 0.15)), 0 + rows * np.tan(random.uniform(-0.15, 0.15))], 
        [cols + cols * np.tan(random.uniform(-0.15, 0.15)), 0 + rows * np.tan(random.uniform(-0.15, 0.15))], 
        [0  + cols * np.tan(random.uniform(-0.15, 0.15)), rows + rows * np.tan(random.uniform(-0.15, 0.15))], 
        [cols + cols * np.tan(random.uniform(-0.15, 0.15)), rows + rows * np.tan(random.uniform(-0.15, 0.15))]
    ])

    M = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply the perspective transform (tilt)
    warped_img = cv2.warpPerspective(img, M, (cols, rows))

    # # Calculate the bounding box of the transformed image
    # x_min, y_min = np.min(dst_points, axis=0)
    # x_max, y_max = np.max(dst_points, axis=0)

    # # Add a small margin to the size of the transparent image
    # margin = 10
    # transparent_height = int(y_max - y_min) + margin
    # transparent_width = int(x_max - x_min) + margin

    # # Calculate the offset for placing the transformed image in the transparent image
    # offset_x = int(abs(x_min)) + margin // 4
    # offset_y = int(abs(y_min)) + margin // 4

    # # Create the transparent image with fully transparent background
    # transparent_img = np.zeros((transparent_height, transparent_width, 4), dtype=np.uint8)
    # transparent_img[:, :, 3] = 0  # Set alpha channel to 0 for fully transparent background

    # # Copy the transformed image into the transparent image with proper offset
    # transparent_img[offset_y:offset_y+warped_img.shape[0], offset_x:offset_x+warped_img.shape[1], :3] = warped_img
    # transparent_img[offset_y:offset_y+warped_img.shape[0], offset_x:offset_x+warped_img.shape[1], 3] = 0

    return warped_img


def place_on_background(img, background, size_range, angle_range, tilt_range):
    # Resize image
    size = random.uniform(*size_range)
    img = cv2.resize(img, None, fx=size, fy=size)

    bk_height, bk_width = background.shape[:2]
    # Add random tilts
    # img = apply_perspective_transform(img, tilt_range)
    img = rotate_and_shear_image(img, bk_height, bk_width)
    # img = stackoverflow_rotate_and_shear_image(img)

    # Ensure the document image is not larger than the background
    if img.shape[0] > background.shape[0] or img.shape[1] > background.shape[1]:
        print("Resized document image is larger than the background. Skipping...")
        return None, None

    # Get random position on background
    max_x = background.shape[1] - img.shape[1]
    max_y = background.shape[0] - img.shape[0]
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)

    # Overlay the document image on the background
    # Overlay the document image on the background
    background[y:y+img.shape[0], x:x+img.shape[1], :3] = img[:, :, :3]

    return background, (x, y, img.shape[1], img.shape[0])

# def place_on_background(img, background, size_range, angle_range, tilt_range):
#     # Resize image
#     size = random.uniform(*size_range)
#     img = cv2.resize(img, None, fx=size, fy=size)

#     # Add random tilts
#     img = apply_perspective_transform(img, tilt_range)

#     # Rotate the image by a random angle
#     angle = random.randint(*angle_range)
#     height, width = img.shape[:2]

#     # Calculate the new image dimensions
#     radians = np.radians(angle)
#     new_width = int(abs(width * np.cos(radians)) + abs(height * np.sin(radians)))
#     new_height = int(abs(width * np.sin(radians)) + abs(height * np.cos(radians)))

#     # Rotate the image
#     M = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
#     rotated_img = cv2.warpAffine(img, M, (new_width, new_height))

#     # Create a larger transparent image
#     larger_img = np.zeros((rotated_img.shape[0]*2, rotated_img.shape[1]*2, 4), dtype=np.uint8)

#     # Place the rotated image in the center of the larger transparent image
#     start_y = larger_img.shape[0] // 4
#     start_x = larger_img.shape[1] // 4
#     larger_img[start_y:start_y+rotated_img.shape[0], start_x:start_x+rotated_img.shape[1], :3] = rotated_img
#     larger_img[start_y:start_y+rotated_img.shape[0], start_x:start_x+rotated_img.shape[1], 3] = 255

#     # Ensure the larger transparent image is not larger than the background
#     if larger_img.shape[0] > background.shape[0] or larger_img.shape[1] > background.shape[1]:
#         print("Resized document image is larger than the background. Skipping...")
#         return None, None

#     # Get random position on background
#     max_x = background.shape[1] - larger_img.shape[1]
#     max_y = background.shape[0] - larger_img.shape[0]
#     x = random.randint(0, max_x)
#     y = random.randint(0, max_y)

#     # Overlay the larger transparent image on the background
#     for c in range(3):
#         background[y:y+larger_img.shape[0], x:x+larger_img.shape[1], c] = larger_img[:,:,c] * (larger_img[:,:,3]/255) +  background[y:y+larger_img.shape[0], x:x+larger_img.shape[1], c] * (1 - larger_img[:,:,3]/255)

#     return background, (x, y, larger_img.shape[1], larger_img.shape[0])


# Paths to folders
doc_folder = 'cropped_documents'
bg_folder = 'backgrounds'

# Loop over all document images
for doc_name in os.listdir(doc_folder):
    doc_path = os.path.join(doc_folder, doc_name)
    print(f"Reading document image from: {doc_path}")  # Debug print
    doc_img = cv2.imread(doc_path)
    if doc_img is None:
        print(f"Failed to read document image from: {doc_path}")
        continue

    # Loop over all background images
    for bg_name in os.listdir(bg_folder):
        bg_path = os.path.join(bg_folder, bg_name)
        print(f"Reading background image from: {bg_path}")  # Debug print
        bg_img = cv2.imread(bg_path)
        if bg_img is None:
            print(f"Failed to read background image from: {bg_path}")
            continue

        # Place document on background
        img, bbox = place_on_background(doc_img, bg_img, (0.3, 0.5), (0, 359), (-0.15, 0.15))

        # Check if image is not None before saving
        if img is not None:
            # Save image and annotation
            cv2.imwrite('dataset/{}_{}.png'.format(doc_name, bg_name), img)
            with open('annotations/{}_{}.txt'.format(doc_name, bg_name), 'w') as f:
                f.write(','.join(map(str, bbox)))
        else:
            print(f"Skipping saving for {doc_name} on {bg_name} due to size mismatch.")

In [4]:
import cv2
import numpy as np
import os
import random



def scale_image(img, bk_height, bk_width):
    # Choose a random scale factor within the specified range
    size = random.uniform(0.3, 0.5)  
    ratio = 1

    if bk_height > bk_width:
        ratio = size * bk_width / img.shape[1]
    else:
        ratio = size * bk_height / img.shape[0]

    # Resize the image
    img = cv2.resize(img, None, fx=ratio, fy=ratio)

    return img

def rotate_and_shear_image(im):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    change from https://stackoverflow.com/a/51109152
    """
    # Randomize the angle, x_shear, y_shear

    angle = random.randint(0, 359)
    x_shear = random.uniform(-0.15, 0.15)
    y_shear = random.uniform(-0.15, 0.15)

    #print(im.shape)
    height, width = im.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_im = cv2.getRotationMatrix2D(image_center, angle, 1.)

    rotation_im[0,1] += x_shear
    rotation_im[1,0] += y_shear

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos_x = abs(rotation_im[0,0]) 
    abs_sin_x = abs(rotation_im[0,1])
    abs_cos_y = abs(rotation_im[1,0]) 
    abs_sin_y = abs(rotation_im[1,1])

    # find the new width and height bounds
    bound_w = int(width * abs_cos_x +  height *  abs_sin_x )
    bound_h = int(height * abs_sin_y  + width * abs_cos_y )

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_im[0, 2] += bound_w/2 - image_center[0] - image_center[1] * x_shear
    rotation_im[1, 2] += bound_h/2 - image_center[1] - image_center[0] * y_shear

    # board_mode = cv2.BORDER_CONSTANT
    # color = (random.randint(230,250), random.randint(230,250), random.randint(230,250))
    
    # # rotate image with the new bounds and translated rotation imrix
    # im = cv2.warpAffine(im, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=board_mode, borderValue=color)
    # Check if image has an alpha channel
    if im.shape[2] < 4:
        # If not, add one
        im = cv2.cvtColor(im, cv2.COLOR_BGR2BGRA)

    # Separate the color and alpha channels
    color = im[..., :3]
    alpha = im[..., 3]

    # Rotate the color channels
    color = cv2.warpAffine(color, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

    # Rotate the alpha channel
    alpha = cv2.warpAffine(alpha, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    # Merge the color and alpha channels
    im = cv2.merge([color, alpha])

    # Define the original corner points
    corners = np.array([[0, 0], [width, 0], [0, height], [width, height]], dtype=np.float32)
    
    # Apply the transformation matrix to the corner points
    transformed_corners = cv2.transform(np.array([corners]), rotation_im)[0]
    print('transformed_corners',transformed_corners)
    #print(rotated_im.shape)
    return im, rotation_im, transformed_corners

def place_on_background(img, background):
    # # Resize image
    # size = random.uniform(*size_range)
    # img = cv2.resize(img, None, fx=size, fy=size)

    bk_height, bk_width = background.shape[:2]
    # scale the image
    img = scale_image(img, bk_height, bk_width)

    # Add random tilts
    # img = apply_perspective_transform(img, tilt_range)
    # img = rotate_and_shear_image(img, bk_height, bk_width)
    img, rotation_im, transformed_corners = rotate_and_shear_image(im=img)
    print('rotation_im',rotation_im)
    # img = stackoverflow_rotate_and_shear_image(img)

    # Ensure the document image is not larger than the background
    if img.shape[0] > background.shape[0] or img.shape[1] > background.shape[1]:
        print("Resized document image is larger than the background. Skipping...")
        return None, None, None

    # Get random position on background
    max_x = background.shape[1] - img.shape[1]
    max_y = background.shape[0] - img.shape[0]
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)

    # Overlay the document image on the background
    for c in range(0, 3):
        background[y:y+img.shape[0], x:x+img.shape[1], c] = img[..., c] * (img[..., 3]/255.0) +  background[y:y+img.shape[0], x:x+img.shape[1], c] * (1.0 - img[..., 3]/255.0)

    # Return the final position and size of the image
    return background, (x, y, img.shape[1], img.shape[0]), transformed_corners


def create_mask(img, bbox, transformed_corners):
    # Initialize an empty mask
    mask = np.zeros_like(img[..., 0])

    # Draw a polygon (document boundary) on the mask
    x, y, w, h = bbox
    corners = np.array([[x + transformed_corners[0][0], y + transformed_corners[0][1]], 
                        [x+transformed_corners[1][0], y + transformed_corners[1][1]], 
                        [x + transformed_corners[3][0], y+transformed_corners[3][1]],
                        [x+transformed_corners[2][0], y+transformed_corners[2][1]]], dtype=np.int32)
    cv2.fillPoly(mask, [corners], 1)

    return mask

# Paths to folders
doc_folder = 'cropped_documents'
bg_folder = 'backgrounds'
mask_folder = 'masks'  # Folder to save masks

# Loop over all document images
for doc_name in os.listdir(doc_folder):
    doc_path = os.path.join(doc_folder, doc_name)
    print(f"Reading document image from: {doc_path}")  # Debug print
    doc_img = cv2.imread(doc_path)
    if doc_img is None:
        print(f"Failed to read document image from: {doc_path}")
        continue

    # Loop over all background images
    for bg_name in os.listdir(bg_folder):
        bg_path = os.path.join(bg_folder, bg_name)
        print(f"Reading background image from: {bg_path}")  # Debug print
        bg_img = cv2.imread(bg_path)
        if bg_img is None:
            print(f"Failed to read background image from: {bg_path}")
            continue

        # Place document on background
        img, bbox, transformed_corners = place_on_background(doc_img, bg_img)

        # Check if image is not None before saving
        if img is not None:
            # Save image and annotation
            cv2.imwrite('dataset/{}_{}.png'.format(doc_name, bg_name), img)
            # Create the mask image
            mask = create_mask(img, bbox, transformed_corners)
            # Save the mask image
            mask_path = os.path.join(mask_folder, '{}_{}.png'.format(doc_name, bg_name))
            cv2.imwrite(mask_path, mask)
        else:
            print(f"Skipping saving for {doc_name} on {bg_name} due to size mismatch.")


Reading document image from: cropped_documents\001_jpg.rf.a831d3c36a427ca0073fbad72a1a6d3c.jpg
Reading background image from: backgrounds\-1x-1.jpg
transformed_corners [[ 1.21588684e+02  3.66081512e+02]
 [-1.70494080e-01  8.23453369e+01]
 [ 3.58170471e+02  2.83654663e+02]
 [ 2.36411316e+02 -8.15124512e-02]]
rotation_im [[-3.42020143e-01  9.81667221e-01  1.21588685e+02]
 [-7.97011760e-01 -3.42020143e-01  3.66081521e+02]]
Reading background image from: backgrounds\0731617fa7e5d174286cf5594ba67fba.jpg
transformed_corners [[ 1.8945657e+02  5.5236549e+01]
 [ 1.0729905e+02  2.0516110e+02]
 [ 8.1700958e+01 -1.6109467e-01]
 [-4.5657349e-01  1.4976346e+02]]
rotation_im [[ -0.46947156  -0.91318322 189.45657181]
 [  0.85671167  -0.46947156  55.23655108]]
Reading background image from: backgrounds\10-tips-to-create-a-perfect-workspace-at-home-2124.jpg
transformed_corners [[ 2.3468866e+02  7.5924048e+02]
 [-2.7919006e-01  1.5845685e+02]
 [ 6.2727917e+02  6.0054309e+02]
 [ 3.9231128e+02 -2.4047852e-

In [8]:
#### Create Negative Samples
def create_negative_mask(img):
    # Initialize an empty mask
    mask = np.zeros_like(img[..., 0])

    # Draw a polygon (document boundary) on the mask
    # x, y, w, h = bbox
    # corners = np.array([[x + transformed_corners[0][0], y + transformed_corners[0][1]], 
    #                     [x+transformed_corners[1][0], y + transformed_corners[1][1]], 
    #                     [x + transformed_corners[3][0], y+transformed_corners[3][1]],
    #                     [x+transformed_corners[2][0], y+transformed_corners[2][1]]], dtype=np.int32)
    # cv2.fillPoly(mask, [corners], 255)

    return mask

# Paths to folders
# doc_folder = 'cropped_documents'
bg_folder = 'negative_temp'
mask_folder = 'negative_masks'  # Folder to save masks

# Loop over all background images
for bg_name in os.listdir(bg_folder):
    bg_path = os.path.join(bg_folder, bg_name)
    print(f"Reading background image from: {bg_path}")  # Debug print
    bg_img = cv2.imread(bg_path)
    if bg_img is None:
        print(f"Failed to read image from: {bg_path}")
    else:
        # Save image and annotation
        cv2.imwrite('negative_dataset/negative_{}.png'.format(bg_name), bg_img)
        # Create the mask image
        mask = create_negative_mask(bg_img)
        # Save the mask image
        mask_path = os.path.join(mask_folder, 'negative_{}.png'.format(bg_name))
        cv2.imwrite(mask_path, mask)

Reading background image from: negative_temp\-1x-1.jpg
Reading background image from: negative_temp\0731617fa7e5d174286cf5594ba67fba.jpg
Reading background image from: negative_temp\10-tips-to-create-a-perfect-workspace-at-home-2124.jpg
Reading background image from: negative_temp\1000_F_246987288_0CNI5GyhHWvSftaO1igXaIfNztJcoVcf.jpg
Reading background image from: negative_temp\135_Auto_68 - Copy.jpg
Reading background image from: negative_temp\135_Auto_68.jpg
Reading background image from: negative_temp\1_4Tm3LoWey4PgJ-8-1KKd9g - Copy.png
Reading background image from: negative_temp\1_4Tm3LoWey4PgJ-8-1KKd9g.png
Reading background image from: negative_temp\1_img_6107cb72d2d9b - Copy.jpg
Reading background image from: negative_temp\1_img_6107cb72d2d9b.jpg
Reading background image from: negative_temp\2016-Colgate-Print-Advertisement-732x1024 - Copy.jpg
Reading background image from: negative_temp\2016-Colgate-Print-Advertisement-732x1024.jpg
Reading background image from: negative_temp\2

In [ ]:
# from datasets import Dataset, DatasetDict, Image

# # your images can of course have a different extension
# # semantic segmentation maps are typically stored in the png format
# image_paths_train = ["path/to/image_1.jpg/jpg", "path/to/image_2.jpg/jpg", ..., "path/to/image_n.jpg/jpg"]
# label_paths_train = ["path/to/annotation_1.png", "path/to/annotation_2.png", ..., "path/to/annotation_n.png"]

# # same for validation
# # image_paths_validation = [...]
# # label_paths_validation = [...]

# def create_dataset(image_paths, label_paths):
#     dataset = Dataset.from_dict({"image": sorted(image_paths),
#                                 "label": sorted(label_paths)})
#     dataset = dataset.cast_column("image", Image())
#     dataset = dataset.cast_column("label", Image())

#     return dataset

# # step 1: create Dataset objects
# train_dataset = create_dataset(image_paths_train, label_paths_train)
# validation_dataset = create_dataset(image_paths_validation, label_paths_validation)

# # step 2: create DatasetDict
# dataset = DatasetDict({
#     "train": train_dataset,
#     "validation": validation_dataset,
#   }
# )

# # step 3: push to hub (assumes you have ran the huggingface-cli login command in a terminal/notebook)
# dataset.push_to_hub("name of repo on the hub")

# # optionally, you can push to a private repo on the hub
# # dataset.push_to_hub("name of repo on the hub", private=True)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
import glob
from datasets import Dataset, DatasetDict, Image

# Get all image and mask paths
image_paths_train_jpg = glob.glob('train_dataset/images/*.jpg')
image_paths_train_png = glob.glob('train_dataset/images/*.png')
image_paths_train = image_paths_train_jpg + image_paths_train_png

label_paths_train_jpg = glob.glob('train_dataset/labels/*.jpg')
label_paths_train_png = glob.glob('train_dataset/labels/*.png')
label_paths_train = label_paths_train_jpg + label_paths_train_png

print(f"Number of images: {len(image_paths_train)}")
print(f"Number of labels: {len(label_paths_train)}")

# validation dataset
image_paths_val_jpg = glob.glob('val_dataset/images/*.jpg')
image_paths_val_png = glob.glob('val_dataset/images/*.png')
image_paths_val = image_paths_val_jpg + image_paths_val_png

label_paths_val_jpg = glob.glob('val_dataset/labels/*.jpg')
label_paths_val_png = glob.glob('val_dataset/labels/*.png')
label_paths_val = label_paths_val_jpg + label_paths_val_png


# Check if the number of images matches the number of labels
assert len(image_paths_train) == len(label_paths_train), "Number of images and labels do not match"

def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "label": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("label", Image())
    return dataset

# step 1: create Dataset objects
train_dataset = create_dataset(image_paths_train, label_paths_train)
val_dataset = create_dataset(image_paths_val, label_paths_val)

# step 2: create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
  }
)

print(dataset)

# step 3: push to hub (assumes you have ran the huggingface-cli login command in a terminal/notebook)
dataset.push_to_hub("narutophuc113/driver_license", private=True)

Number of images: 200
Number of labels: 200
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 58
    })
})


Uploading the dataset shards: 100%|██████████| 1/1 [00:50<00:00, 50.86s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/narutophuc113/driver_license/commit/b544f4bc81b9c501352b0bbd0be138f0f8659390', commit_message='Upload dataset', commit_description='', oid='b544f4bc81b9c501352b0bbd0be138f0f8659390', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
import json
# simple example
id2label = {0: 'background', 1: 'document'}
with open('id2label.json', 'w') as fp:
  json.dump(id2label, fp)

In [12]:
from datasets import load_dataset

hf_dataset_identifier = "narutophuc113/driver_license"
ds = load_dataset(hf_dataset_identifier)

# update column names
ds = ds.rename_column("image", "pixel_values")

Generating validation split: 100%|██████████| 58/58 [00:00<00:00, 195.89 examples/s]


In [ ]:
ds = ds.shuffle(seed=1)
ds = ds["train"].train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]

In [ ]:
import json
from huggingface_hub import hf_hub_download

repo_id = f"datasets/{hf_dataset_identifier}"
filename = "id2label.json"
id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

In [ ]:
from torchvision.transforms import ColorJitter
from transformers import SegformerImageProcessor

processor = SegformerImageProcessor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = processor(images, labels)
    return inputs


# Set transforms
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [ ]:
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0" 
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)

In [ ]:
from transformers import TrainingArguments

epochs = 50
lr = 0.00006
batch_size = 2

hub_model_id = "segformer-b0-finetuned-segments-driver-license"

training_args = TrainingArguments(
    "segformer-b0-finetuned-segments-driver-license-output",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_strategy="end",
)

In [ ]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric.compute(
        predictions=pred_labels,
        references=labels,
        num_labels=len(id2label),
        ignore_index=0,
        reduce_labels=processor.do_reduce_labels,
    )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    keys = list(id2label.keys())
    metrics.update({f"accuracy_{id2label[keys[i]]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[keys[i]]}": v for i, v in enumerate(per_category_iou)})
    # metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    # metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})
    
    return metrics

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

                                                     

                                           
 90%|████████▉ | 2597/2900 [3:13:10<15:29,  3.07s/it]


{'loss': 0.0009, 'grad_norm': 0.009939565323293209, 'learning_rate': 6.26896551724138e-06, 'epoch': 44.78}


                                                     

                                           
 90%|████████▉ | 2598/2900 [3:13:13<15:00,  2.98s/it]


{'loss': 0.0013, 'grad_norm': 0.013795644976198673, 'learning_rate': 6.248275862068965e-06, 'epoch': 44.79}


                                                     

                                           
 90%|████████▉ | 2599/2900 [3:13:16<15:40,  3.12s/it]


...































C:\Users\RazerBook\AppData\Roaming\Python\Python311\site-packages\datasets\features\image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
C:\Users\RazerBook\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0\mean_iou.py:258: RuntimeWarning: invalid value encountered in scalar divide
  all_acc = total_area_intersect.sum() / total_area_label.sum()
C:\Users\RazerBook\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0\mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
C:\Users\RazerBook\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\9e450724f21f

{'eval_loss': 0.0010219422401860356, 'eval_mean_iou': 0.0, 'eval_mean_accuracy': nan, 'eval_overall_accuracy': nan, 'eval_accuracy_background': nan, 'eval_accuracy_unknown': nan, 'eval_accuracy_document': nan, 'eval_iou_background': 0.0, 'eval_iou_unknown': 0.0, 'eval_iou_document': 0.0, 'eval_runtime': 9.6475, 'eval_samples_per_second': 3.006, 'eval_steps_per_second': 1.555, 'epoch': 50.0}


                                                     

                                           
100%|██████████| 2900/2900 [3:35:35<00:00,  4.46s/it]

{'train_runtime': 12935.6556, 'train_samples_per_second': 0.448, 'train_steps_per_second': 0.224, 'train_loss': 0.02290498763904907, 'epoch': 50.0}


TrainOutput(global_step=2900, training_loss=0.02290498763904907, metrics={'train_runtime': 12935.6556, 'train_samples_per_second': 0.448, 'train_steps_per_second': 0.224, 'total_flos': 1.01669175558144e+17, 'train_loss': 0.02290498763904907, 'epoch': 50.0})

In [22]:
kwargs = {
    "tags": ["vision", "image-segmentation"],
    "finetuned_from": pretrained_model_name,
    "dataset": hf_dataset_identifier,
}

processor.push_to_hub(hub_model_id)
trainer.push_to_hub(**kwargs)



model.safetensors:   0%|          | 0.00/14.9M [00:00<?, ?B/s]


model.safetensors:   0%|          | 16.4k/14.9M [00:00<13:09, 18.8kB/s]


training_args.bin: 100%|██████████| 5.05k/5.05k [00:01<00:00, 4.90kB/s]
model.safetensors: 100%|██████████| 14.9M/14.9M [00:10<00:00, 1.36MB/s]


Upload 2 LFS files: 100%|██████████| 2/2 [00:11<00:00,  5.74s/it]


CommitInfo(commit_url='https://huggingface.co/narutophuc113/segformer-b0-finetuned-segments-driver-license/commit/9f192e267b9dd197bae9b7cf91b84bb518a07609', commit_message='End of training', commit_description='', oid='9f192e267b9dd197bae9b7cf91b84bb518a07609', pr_url=None, pr_revision=None, pr_num=None)